In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [ ]:
base_url = 'https://mogi.vn/ha-noi/mua-nha-dat'
respone = requests.get(base_url)


In [ ]:
# Links = []
# for page in range(1,21):
#     url = f"{base_url}?&cp={page}"
#     print(f"Đang cào trang {page}: {url}")
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')

#         # Tìm tất cả thẻ a có class="link-overlay"
#         links = soup.find_all('a', class_='link-overlay')

#         # In ra href của từng thẻ
#         for link in links:
#             href = link.get('href')
#             Links.append(href)
#             # print(href)
#     else:
        # print("Không thể truy cập trang:", response.status_code)

 

Đang cào trang 1: https://mogi.vn/ha-noi/mua-nha-dat?&cp=1
Đang cào trang 2: https://mogi.vn/ha-noi/mua-nha-dat?&cp=2
Đang cào trang 3: https://mogi.vn/ha-noi/mua-nha-dat?&cp=3
Đang cào trang 4: https://mogi.vn/ha-noi/mua-nha-dat?&cp=4
Đang cào trang 5: https://mogi.vn/ha-noi/mua-nha-dat?&cp=5
Đang cào trang 6: https://mogi.vn/ha-noi/mua-nha-dat?&cp=6
Đang cào trang 7: https://mogi.vn/ha-noi/mua-nha-dat?&cp=7
Đang cào trang 8: https://mogi.vn/ha-noi/mua-nha-dat?&cp=8
Đang cào trang 9: https://mogi.vn/ha-noi/mua-nha-dat?&cp=9
Đang cào trang 10: https://mogi.vn/ha-noi/mua-nha-dat?&cp=10
Đang cào trang 11: https://mogi.vn/ha-noi/mua-nha-dat?&cp=11
Đang cào trang 12: https://mogi.vn/ha-noi/mua-nha-dat?&cp=12
Đang cào trang 13: https://mogi.vn/ha-noi/mua-nha-dat?&cp=13
Đang cào trang 14: https://mogi.vn/ha-noi/mua-nha-dat?&cp=14
Đang cào trang 15: https://mogi.vn/ha-noi/mua-nha-dat?&cp=15
Đang cào trang 16: https://mogi.vn/ha-noi/mua-nha-dat?&cp=16
Đang cào trang 17: https://mogi.vn/ha-noi/

In [4]:
import csv
Link_list = []
data_list = []
with open('mogi_links.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['link'])

    for page in range(1, 400):
        url = f"{base_url}?&cp={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all("a", class_="link-overlay")
        for link in links:
            href = link.get("href")
            if href:
                writer.writerow([href])
                Link_list.append(href)


for link in Link_list:
    res = requests.get(link)
    if res.status_code == 200:
        data = {}
        sop = BeautifulSoup(res.text,"html.parser")
        infos = sop.find_all("div", class_ = "info-attr clearfix")
        price = sop.find('div',class_ = "price")
        address = sop.find('div',class_ = "address")
        data['price'] = price.get_text(strip = True)
        data['address'] = address.get_text(strip = True)
        for info in infos:
            span = info.find_all('span')
            if len(span) >= 2:
                key = span[0].get_text(strip=True)
                value = span[1].get_text(strip=True)
                data[key] = value
        data_list.append(data)
        time.sleep(random.uniform(1,2))

fieldnames = set()
for d in data_list:
    fieldnames.update(d.keys())
fieldnames = list(fieldnames)

with open('batdongsan.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_list)

print("Đã lưu dữ liệu vào file batdongsan.csv")

KeyboardInterrupt: 

Số dòng ban đầu: 4492
Số dòng sau khi loại bỏ trùng lặp: 4488
Số dòng trùng lặp đã xóa: 4
Tỷ lệ trùng lặp: 0.09%


In [6]:
import requests
import csv
import time
import random
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pathlib import Path

BASE_LIST = "https://mogi.vn/ha-noi/mua-nha-dat"
BASE = "https://mogi.vn"
TOTAL_PAGES = 400
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}
SESSION = requests.Session()
SESSION.headers.update(HEADERS)

CHECKPOINT_DIR = Path(".checkpoints")
CHECKPOINT_DIR.mkdir(exist_ok=True)
PAGE_CKPT = CHECKPOINT_DIR / "pages.txt"
DETAIL_CKPT = CHECKPOINT_DIR / "detail.txt"
LINKS_FILE = Path("mogi_links.csv")
DATA_FILE = Path("batdongsan.csv")


def request_with_retry(url: str, max_retries: int = 5, base_delay: float = 1.0, timeout: int = 15):
    last_exc = None
    for attempt in range(max_retries):
        try:
            resp = SESSION.get(url, timeout=timeout)
            if resp.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"Transient {resp.status_code}")
            resp.raise_for_status()
            return resp
        except Exception as exc:
            last_exc = exc
            sleep_s = base_delay * (2 ** attempt) + random.uniform(0, 0.5)
            time.sleep(sleep_s)
    raise last_exc


def read_checkpoint(path: Path, default_value: int) -> int:
    if path.exists():
        try:
            return int(path.read_text().strip())
        except Exception:
            return default_value
    return default_value


def write_checkpoint(path: Path, value: int) -> None:
    path.write_text(str(value))


def ensure_csv_header(path: Path, fieldnames):
    if not path.exists() or path.stat().st_size == 0:
        with path.open("w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()


# 1) Crawl listing pages -> collect detail links (dedup, incremental)
start_page = read_checkpoint(PAGE_CKPT, 1)
seen_links = set()
if LINKS_FILE.exists():
    with LINKS_FILE.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            href = row.get("link")
            if href:
                seen_links.add(href)

with LINKS_FILE.open("a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if LINKS_FILE.stat().st_size == 0:
        writer.writerow(["link"])  # header

    for page in range(start_page, TOTAL_PAGES + 1):
        list_url = f"{BASE_LIST}?&cp={page}"
        try:
            resp = request_with_retry(list_url)
            soup = BeautifulSoup(resp.text, "html.parser")
            new_count = 0
            for a in soup.find_all("a", class_="link-overlay"):
                href = a.get("href")
                if not href:
                    continue
                full_url = urljoin(BASE, href)
                if full_url not in seen_links:
                    seen_links.add(full_url)
                    writer.writerow([full_url])
                    new_count += 1
            print(f"Trang {page}: thêm {new_count} link (tổng link duy nhất: {len(seen_links)})")
            write_checkpoint(PAGE_CKPT, page + 1)
            time.sleep(random.uniform(0.5, 1.2))
        except Exception as e:
            print(f"Trang {page} lỗi: {e}")
            # vẫn tăng nhẹ delay rồi tiếp
            time.sleep(random.uniform(2, 3))


# 2) Crawl detail pages -> write rows incrementally
# Prepare fieldnames with a minimal stable set; will expand dynamically if needed
base_fields = ["url", "price", "address"]
ensure_csv_header(DATA_FILE, base_fields)

# Load all links and resume index
all_links = []
with LINKS_FILE.open("r", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        href = row.get("link")
        if href:
            all_links.append(href)

start_index = read_checkpoint(DETAIL_CKPT, 0)

# Read existing headers to preserve columns
with DATA_FILE.open("r", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    try:
        current_headers = next(reader)
    except StopIteration:
        current_headers = base_fields

from concurrent.futures import ThreadPoolExecutor, as_completed

CHUNK_SIZE = 50
MAX_WORKERS = 8


def parse_detail(detail_url: str):
    try:
        resp = request_with_retry(detail_url, max_retries=4)
        soup = BeautifulSoup(resp.text, "html.parser")
        price_tag = soup.find('div', class_='price')
        address_tag = soup.find('div', class_='address')
        info_blocks = soup.find_all('div', class_='info-attr clearfix')
        row = {
            "url": detail_url,
            "price": price_tag.get_text(strip=True) if price_tag else "",
            "address": address_tag.get_text(strip=True) if address_tag else "",
        }
        for block in info_blocks:
            spans = block.find_all('span')
            if len(spans) >= 2:
                key = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                if key and value:
                    row[key] = value
        return row
    except Exception as e:
        return {"url": detail_url, "error": str(e)}


for start in range(start_index, len(all_links), CHUNK_SIZE):
    end = min(start + CHUNK_SIZE, len(all_links))
    batch = all_links[start:end]
    rows = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_url = {executor.submit(parse_detail, u): u for u in batch}
        for fut in as_completed(future_to_url):
            row = fut.result()
            if row is None:
                continue
            rows.append(row)

    # Expand headers if needed
    keys_in_batch = set()
    for r in rows:
        keys_in_batch.update(r.keys())
    if any(k not in current_headers for k in keys_in_batch):
        new_headers = list(current_headers)
        for k in keys_in_batch:
            if k not in new_headers:
                new_headers.append(k)
        existing_rows = []
        if DATA_FILE.exists() and DATA_FILE.stat().st_size > 0:
            with DATA_FILE.open("r", newline="", encoding="utf-8") as rf:
                r = csv.DictReader(rf)
                existing_rows.extend(r)
        with DATA_FILE.open("w", newline="", encoding="utf-8") as wf:
            w = csv.DictWriter(wf, fieldnames=new_headers)
            w.writeheader()
            for rrow in existing_rows:
                w.writerow(rrow)
        current_headers = new_headers

    # Append batch
    with DATA_FILE.open("a", newline="", encoding="utf-8") as wf:
        w = csv.DictWriter(wf, fieldnames=current_headers)
        for r in rows:
            w.writerow(r)

    write_checkpoint(DETAIL_CKPT, end)
    print(f"Chi tiết {end}/{len(all_links)} (batch {start}-{end-1})")
    time.sleep(random.uniform(0.8, 1.5))

print("Hoàn tất (hoặc checkpoint đã đến cuối danh sách)")


Trang 52: thêm 0 link (tổng link duy nhất: 6037)
Trang 53: thêm 0 link (tổng link duy nhất: 6037)
Trang 54: thêm 0 link (tổng link duy nhất: 6037)
Trang 55: thêm 0 link (tổng link duy nhất: 6037)
Trang 56: thêm 0 link (tổng link duy nhất: 6037)
Trang 57: thêm 0 link (tổng link duy nhất: 6037)
Trang 58: thêm 0 link (tổng link duy nhất: 6037)
Trang 59: thêm 0 link (tổng link duy nhất: 6037)


KeyboardInterrupt: 